In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# === Data Loading and Preprocessing ===
def load_and_preprocess_data(base_path='D:/DL_Project/', file_ids=range(1, 20)):
    all_eeg_data, all_labels = [], []
    for fid in file_ids:
        file_num = str(fid)
        preprocessed_file = f'{base_path}eeg_preprocessed/{file_num}.mat'
        features_file = f'{base_path}eeg_features/{file_num}.mat'
        labels_file = f'{base_path}continuous_labels/{file_num}.mat'

        # Load .mat files
        preprocessed_data = scipy.io.loadmat(preprocessed_file)
        features_data = scipy.io.loadmat(features_file)
        labels_data = scipy.io.loadmat(labels_file)

        print(f"Processing file set {file_num}:")
        print("Keys in Preprocessed File:", preprocessed_data.keys())
        print("Keys in Features File:", features_data.keys())
        print("Keys in Labels File:", labels_data.keys())

        # Load EEG features (differential entropy 'de_1' to 'de_80')
        eeg_features = []
        fixed_time_segments = 32  # Increased from 15 to 32
        for key in range(1, 81):
            key_str = f'de_{key}'
            if key_str in features_data:
                trial_data = features_data[key_str]
                print(f"Shape of trial {key} ('{key_str}'): {trial_data.shape}")

                # Ensure shape is (time_segments, bands, channels=62)
                if trial_data.shape[2] == 62:
                    pass
                elif trial_data.shape[0] == 62:
                    trial_data = trial_data.transpose(2, 1, 0)
                elif trial_data.shape[1] == 62:
                    trial_data = trial_data.transpose(2, 0, 1)
                else:
                    raise ValueError(f"Unexpected shape for trial {key}: {trial_data.shape}")

                # Pad or crop to fixed_time_segments
                current_time_segments = trial_data.shape[0]
                if current_time_segments > fixed_time_segments:
                    trial_data = trial_data[:fixed_time_segments, :, :]
                elif current_time_segments < fixed_time_segments:
                    pad_width = fixed_time_segments - current_time_segments
                    trial_data = np.pad(trial_data, ((0, pad_width), (0, 0), (0, 0)), mode='constant')
                
                eeg_features.append(trial_data)

        if not eeg_features:
            raise ValueError(f"No EEG features found in {features_file}")
        eeg_features = np.array(eeg_features)
        print(f"EEG features shape for file {file_num}: {eeg_features.shape}")

        # Load and process labels
        labels = []
        for key in range(1, 81):
            key_str = str(key)
            if key_str in labels_data:
                label = labels_data[key_str]
                current_time_segments = label.shape[1] if label.ndim > 1 else 1
                if current_time_segments > fixed_time_segments:
                    label = label[:, :fixed_time_segments]
                elif current_time_segments < fixed_time_segments:
                    pad_width = fixed_time_segments - current_time_segments
                    label = np.pad(label, ((0, 0), (0, pad_width)), mode='constant')
                labels.append(label)

        if not labels:
            raise ValueError(f"No labels found in {labels_file}")
        labels = np.array(labels)
        print(f"Raw labels shape for file {file_num}: {labels.shape}")

        # Average labels over time segments
        if labels.ndim == 3:
            labels = np.mean(labels, axis=2)
        elif labels.ndim == 2:
            labels = np.mean(labels, axis=1, keepdims=True)
        labels = labels.squeeze()

        # Map continuous labels to discrete emotions
        discrete_labels = np.zeros(labels.shape[0], dtype=int)
        for i, label in enumerate(labels):
            label = label / 100.0 if label > 10 else label / 10.0  # Normalize
            if label <= 0.2:
                discrete_labels[i] = 0  # Neutral
            elif 0.2 < label <= 0.4:
                discrete_labels[i] = 1  # Sad
            elif 0.4 < label <= 0.6:
                discrete_labels[i] = 2  # Happy
            elif 0.6 < label <= 0.8:
                discrete_labels[i] = 3  # Fear
            else:
                discrete_labels[i] = 4  # Angry
        labels = discrete_labels
        print(f"Processed labels shape for file {file_num}: {labels.shape}, Unique labels: {np.unique(labels)}")

        all_eeg_data.append(eeg_features)
        all_labels.append(labels)

    # Concatenate across files
    all_eeg_data = np.concatenate(all_eeg_data, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    print(f"Total EEG data shape: {all_eeg_data.shape}, Total labels shape: {all_labels.shape}")

    # Reshape for CNN (32x64)
    n_trials = all_eeg_data.shape[0]
    eeg_data = []
    for trial in range(n_trials):
        trial_data = all_eeg_data[trial]
        trial_data = np.mean(trial_data, axis=1)  # Average over bands, shape: (time_segments, 62)
        # Pad or crop time segments to 32
        if trial_data.shape[0] > 32:
            trial_data = trial_data[:32, :]
        elif trial_data.shape[0] < 32:
            trial_data = np.pad(trial_data, ((0, 32 - trial_data.shape[0]), (0, 0)), mode='constant')
        # Pad channels to 64
        if trial_data.shape[1] < 64:
            trial_data = np.pad(trial_data, ((0, 0), (0, 64 - trial_data.shape[1])), mode='constant')
        eeg_data.append(trial_data)

    eeg_data = np.array(eeg_data)
    print(f"Reshaped EEG data shape: {eeg_data.shape}")  # Should be (n_trials, 32, 64)

    # Normalize
    scaler = StandardScaler()
    eeg_data_flat = eeg_data.reshape(eeg_data.shape[0], -1)
    eeg_data = scaler.fit_transform(eeg_data_flat).reshape(eeg_data.shape[0], 32, 64)

    # Balance classes with SMOTE
    smote = SMOTE(random_state=42)
    eeg_data_flat = eeg_data.reshape(eeg_data.shape[0], -1)
    eeg_data_resampled, labels_resampled = smote.fit_resample(eeg_data_flat, all_labels)
    eeg_data = eeg_data_resampled.reshape(-1, 32, 64)
    print(f"Resampled data shape: {eeg_data.shape}, Labels shape: {labels_resampled.shape}")
    print(f"Class distribution: {dict(zip(*np.unique(labels_resampled, return_counts=True)))}")

    return eeg_data, labels_resampled

# === Improved Hybrid Model ===
class ImprovedHybridModel(nn.Module):
    def __init__(self, num_classes=5):
        super(ImprovedHybridModel, self).__init__()
        # CNN layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)  # Output: (32, 16, 32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)  # Output: (64, 8, 16)
        # LSTM
        self.hidden_size = 256
        self.lstm = nn.LSTM(64 * 16, self.hidden_size, num_layers=2, batch_first=True, bidirectional=True, dropout=0.4)
        self.attention = Attention(self.hidden_size * 2)
        # Fully connected layers
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(self.hidden_size * 2, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool1(self.relu(self.bn1(self.conv1(x))))  # (batch, 32, 16, 32)
        x = self.pool2(self.relu(self.bn2(self.conv2(x))))  # (batch, 64, 8, 16)
        x = x.view(-1, 8, 64 * 16)  # (batch, 8, 1024)
        lstm_out, (hn, _) = self.lstm(x)
        hn = torch.cat((hn[-2], hn[-1]), dim=1)  # Concatenate bidirectional hidden states
        context = self.attention(hn, lstm_out)
        out = self.dropout(self.relu(context))
        out = self.fc(out)
        return out

# === Attention Mechanism ===
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        stdv = 1. / (self.hidden_size ** 0.5)
        self.v.data.uniform_(-stdv, stdv)

    def forward(self, hidden, encoder_outputs):
        batch_size, seq_len, _ = encoder_outputs.size()
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy.matmul(self.v)
        attn_weights = torch.softmax(energy, dim=1).unsqueeze(2)
        context = (attn_weights * encoder_outputs).sum(dim=1)
        return context

# === Training and Evaluation with Mixup ===
def train_and_evaluate(model, train_loader, test_loader, num_epochs=100, device='cuda' if torch.cuda.is_available() else 'cpu', alpha=1.0):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Increased weight decay
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=15, T_mult=2, eta_min=1e-6)

    train_losses, val_losses = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            # Apply mixup with 50% probability
            if np.random.rand() < 0.5:
                lam = np.random.beta(alpha, alpha)
                index = torch.randperm(inputs.size(0)).to(device)
                mixed_inputs = lam * inputs + (1 - lam) * inputs[index]
                outputs = model(mixed_inputs)
                loss = lam * criterion(outputs, labels) + (1 - lam) * criterion(outputs, labels[index])
            else:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0.0
        y_true, y_pred = [], []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
                _, predicted = torch.max(outputs, 1)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())
        val_loss = val_loss / len(test_loader)
        val_losses.append(val_loss)

        scheduler.step(epoch + 1)
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    cm = confusion_matrix(y_true, y_pred)

    return train_losses, val_losses, accuracy, precision, recall, f1, cm

# === Main Execution ===
eeg_data, labels = load_and_preprocess_data()
X_train, X_test, y_train, y_test = train_test_split(eeg_data, labels, test_size=0.2, random_state=42)

# Prepare data for CNN and Hybrid
X_train_cnn = torch.FloatTensor(X_train).unsqueeze(1)  # Shape: (n_train, 1, 32, 64)
X_test_cnn = torch.FloatTensor(X_test).unsqueeze(1)    # Shape: (n_test, 1, 32, 64)
y_train_t = torch.LongTensor(y_train)
y_test_t = torch.LongTensor(y_test)

# DataLoaders
train_dataset_cnn = TensorDataset(X_train_cnn, y_train_t)
test_dataset_cnn = TensorDataset(X_test_cnn, y_test_t)
train_loader_cnn = DataLoader(train_dataset_cnn, batch_size=32, shuffle=True)
test_loader_cnn = DataLoader(test_dataset_cnn, batch_size=32)

# Models (focusing on Hybrid)
models = {
    'Hybrid': ImprovedHybridModel()
}

# Training and evaluation
results = {}
for name, model in models.items():
    print(f"\nTraining {name} Model...")
    train_loader = train_loader_cnn
    test_loader = test_loader_cnn
    train_losses, val_losses, acc, prec, rec, f1, cm = train_and_evaluate(model, train_loader, test_loader, alpha=1.0)
    results[name] = {'train_losses': train_losses, 'val_losses': val_losses, 'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1, 'confusion_matrix': cm}
    print(f"{name} - Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")

# === Visualization ===
plt.figure(figsize=(10, 5))
plt.plot(results['Hybrid']['train_losses'], label='Train Loss')
plt.plot(results['Hybrid']['val_losses'], label='Val Loss')
plt.title('Hybrid Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()